# Bereiten wir das Script vor

In [1]:
#Imports
import os #amcht Liste aus allen Dokumenten, die in einem Folder sind > nützlich, um files in Serie zu bearbeiten 
import tqdm #Fortschritt der Berechnung anzeign (für aufwendige Operationen)
import textract # zieht Text aus PDFs
import pandas as pd

## Vorbereitung

In [2]:
#Als Vorbereitung, machen aus dem Inhalt des Folders eine Liste. Es handelt sich hierbei
#um eine Liste aller pdfs im Ordner pdfs. Es sind die Swissmedic Recalls (#ImplantFiles)
pdfs = os.listdir('pdfs')
#os.listdir() macht eine Liste der Files im Folder Folders, dessen Pfad in der Klammer() angegeben ist  

In [3]:
#erste 10 Einträge der Liste anzeigen:
pdfs[0:10]

['Vk_20180514_49documents0.pdf',
 'Vk_20180522_02documents1.pdf',
 'Vk_20180413_17documents3.pdf',
 'Vk_20180525_03documents1.pdf',
 'Vk_20180529_02documents2.pdf',
 'Vk_20180517_02documents2.pdf',
 'Vk_20180517_02documents3.pdf',
 'Vk_20180525_03documents0.pdf',
 'Vk_20180413_17documents2.pdf',
 'Vk_20180522_02documents0.pdf']

## Fortschritt anzeigen

In [4]:
#hier wird das Fortschritts-Anzeige-Tool tqdm geladen
from tqdm import tqdm

#wenn man dann tqdm an dieser Stelle in den Loop einbaut, zeigt er den Fortschritt an
for i in tqdm(range(10000000)): #100000000000
    elem = i

100%|██████████| 10000000/10000000 [00:04<00:00, 2411300.30it/s]


## Ein File auslesen

In [5]:
# textract: häufigster Befehl ist .process, in Klammer der Pfad zum zu verarbeitende File 
#(hier unter Verwendung der zuvor generierten PDF-Dateien-Liste, Pfad-Anfang + Datei[x])
text = textract.process("pdfs/"+pdfs[1]) 
#falls es nicht klappt, die Klammer  ergänzen: textract.process("pdfs/"+pdfs[1], method='pdfminer', encoding='ISO-8859-1')
#method='pdfminer' = nach Text suchen, 'tesseract' wäre Bildersuche
#encoding='ISO-8859-1' = Französische Sonderzeichen wie é usw. richtig lesen, andere ISO im Netz

In [6]:
#nweil das Resultat den Typ "Bite" hat, muss mand das Format in String umwandeln – und schon ist es lesbar:
str(text)

'b"Schuchmann GmbH & Co. KG \\xc2\\xb7 D\\xc3\\xbctestra\\xc3\\x9fe 3 \\xc2\\xb7 49205 Hasbergen\\n\\nHasbergen, le 25/05/2018\\nInformation urgente relative \\xc3\\xa0 la s\\xc3\\xa9curit\\xc3\\xa9 \\xe2\\x80\\x93 Retrait/remplacement support de poign\\xc3\\xa9e\\nMALTE (T. 2-4), MALTE Outdoor (T. 2+3) et MARCY (T. 2+3)\\nMesdames, Messieurs,\\nnous avons pour principe de livrer des produits de qualit\\xc3\\xa9 sup\\xc3\\xa9rieure et donc conformes aux\\nexigences de qualit\\xc3\\xa9 de fabrication et de s\\xc3\\xa9curit\\xc3\\xa9 de production. C\\xe2\\x80\\x99est pourquoi nos produits\\nsont soumis en permanence \\xc3\\xa0 des tests effectu\\xc3\\xa9s en interne par notre entreprise mais \\xc3\\xa9galement par des laboratoires de contr\\xc3\\xb4le accr\\xc3\\xa9dit\\xc3\\xa9s selon les normes en vigueur.\\nCes tests simulent les sollicitations auxquelles les produits seront soumis plus tard et nous\\ndonnent, en tant que fabricant, l\\xe2\\x80\\x99assurance d\\xc3\\xa9\\xef\\xac\\x8

## Alle Files auslesen

In [7]:
lst = []
mistakes = []
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        lst.append(text)
    except:
        mistakes.append(pdf)

100%|██████████| 82/82 [02:38<00:00,  1.93s/it]


In [8]:
mistakes

['Vk_20180517_12documents0.pdf']

## Bilder auslesen und in Text umwandeln

In [9]:
text = textract.process("pdfs/"+pdfs[0], method='tesseract') #Nicht ideal, aber gut

In [10]:
text

b'dical\n\nWassenburg Medical B.V.\nEdisonnng 9\n\n6669 NA DODEWAARD\nTHE NETHERLANDS\n\nT +31 (0)488 700 500\n\nF +31 (0)488 453 685\n\n| DRINGEND: SICHERHEITSHINWE See"\n\nRUCKRUF VON KANALVERBINDUNGS-\n\nMATERIAL\nWASSENBURG\xc2\xae REINIGUNGS- UND DESINFEKTIONSGERAT\nENDOSKOPIE\n\nPRODUKTNAME: WASSENBURG\xc2\xae WD440, WD440 PT, WD415 Reinigungs- und\nDesinfektionsgerat Endoskopie\n\nDATUM: 22. Mai 2018\n\nHANDLUNGSTYP: _ Sicherheitsmassnahme\n\nZU HANDEN VON:  Abteilungsleiter fir Dekontamination / Endoskopie\n\nAngaben zu betroffenen Geraten:\n\nBei einigen Chargen der Kanalverbindungen fur die Verbindung von Endoskopen in WASSENBURG\xc2\xae\nEndoskop-Reinigungs- und Desinfektionsgeraten wurde ein Defekt festgestellt, infolgedessen einige\nVerbindungsstucke wahrend der Endoskopaufbereitung auseinanderfallen k\xc3\xa9nnen,\n\nDie betroffenen Artikel und Chargen k\xc3\xa9nnen der folgenden Tabelle entnommen werden,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\

## Alles zusammen

In [11]:
#Rohe Analyse
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        #hiermit haue ich die "\n"-Zeielnumbrüche raus aus dem Text
        #man könnte auch schreiben text = str(text).replace("\n", " "), abr diese Variante ist sicherer
        text = text.decode("ISO-8859-1").replace("\n", " ")
        if len(text) < 40:
        # weil ich festgestellt habe, dass die kürzeren Bilder sind
            #wenn man weiss, welche Sprache, kann man hier auch language = "deu" verwenden statt encoding "ISO..."
            text = textract.process("pdfs/"+pdf, method='tesseract', encoding ="ISO-8859-1")
            text = text.decode("ISO-8859-1").replace("\n", " ")
            
        mini_dict = {'Text':text,
                     'File':pdf}
        #in diesen Minidict trage ich jeweils den Titel und den File-Inhalt ein
        
        lst.append(mini_dict)
    except:
        mini_dict = {'Text':'Fehlermeldung',
                     'File':pdf}
        lst.append(mini_dict)

100%|██████████| 82/82 [04:43<00:00,  3.46s/it]


In [17]:
df = pd.DataFrame(lst)
df.dtypes

0    object
dtype: object

In [ ]:
# Jetzt kann ich zum Beispiel eine Spalte machen, in der ich die Länger der einzelnen Texte anschreibe:
#hier die Funktion, die die Länge von jedem eingegebenen Element misst und sie rausspuckt
def lengt(elem)
    elem = len(elem)
    return elem

df["Textlänge"] = df["Text"].apply(length)

In [13]:
#Erste Säuberung
df = pd.DataFrame(lst)
def date(elem):
    elem = elem[3:11]
    return elem
df['date'] = df['File'].apply(date)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.index = df['date']
#Suchen wir noch nach Implantaten
df['implant'] = df['Text'].str.contains('implant')
df.to_csv('datafile.csv')

KeyError: 'File'